This notebook continuously estimates the prevailing, realtime, positivity values via 

* $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$

curves, wherein

$\qquad \qquad \qquad positives_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{P_{\delta}}  \right) \times \frac{1}{population}$


$\qquad \qquad \qquad \qquad  tests_{c, \small{100K}, \tau} \; = \; \small{100\,000} \times \displaystyle \left(\sum_{\substack{
   start \\
   date
  }}^{\tau}{T_{\delta}} \right) \times \frac{1}{population}$  

<br>

which are the positive cases per hundred thousand and test numbers per hundred thousand, respectively, based on their cumulative values and

<br>

<table style="width:35%; text-align: left; border: 0px solid black; float:left; margin-left: 50px">
    <tr>
        <th style="width:20%;">$\mathcal{Variable} \qquad$</th><th></th> 
    </tr>
    <tr>
        <td>$\tau$</td><td>end date</td>
    </tr>
    <tr>
        <td>$\delta$</td><td>a series date</td>
    </tr>
    <tr>
      <td>$P_{\delta}$</td><td>The number of positive cases on date $\delta$.</td>
    </tr>
    <tr>
      <td>$T_{\delta}$</td><td>The number of tests on date $\delta$.</td>
    </tr>
    
</table>


<br>

<br>

The subscript $c$ denotes cumulative.   The graph image below illustrates the curves in question.  In brief 

* The quotient of the coordinates of a point along a curve is the cumulative positivity value. 

* The gradient of a modelled secant line to a sequential set of curve rear points estimates the prevailing positivity value.

* The aforementioned secant line model predicts the expected values of $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}}$ increases.


<div style="margin-left: 100px;">
    <img src="https://github.com/briefings/sars/raw/develop/fundamentals/atlantic/docs/ptr.png" longdesc="surveillance" width="40%" style="float:center">
</div>

<br>

Upcoming: Link this image to the interactive graph.



<br>

## Preliminaries

### Libraries

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
import dask

import logging

import os
import pathlib
import sys

import statsmodels.regression.linear_model


In [2]:
from statsmodels.formula.api import ols

<br>

### Paths

In [3]:
child = os.getcwd()
parent = str(pathlib.Path(child).parent)

In [4]:
root = os.path.join(child, 'warehouse')
warehouse = os.path.join(root, 'prospects')

<br>

Appending paths

In [5]:
sys.path.append(parent)

<br>

### Logging

In [6]:
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

<br>

### Custom

Import

In [7]:
import cartographs.boundaries.us.boundaries
import cartographs.boundaries.us.settings

In [8]:
import algorithms.curves.estimates
import algorithms.curves.predictions
import algorithms.curves.secants

In [9]:
import atlantic.base.directories
import atlantic.investigations.prospects
import atlantic.investigations.hospitalizations

<br>

Set-up directories

In [10]:
directories = atlantic.base.directories.Directories()
directories.cleanup(listof=[warehouse])
directories.create(listof=[warehouse])

<br>
<br>

### Classes

**Boundaries**

In [11]:
settings = cartographs.boundaries.us.settings.Settings()
boundaries = cartographs.boundaries.us.boundaries.Boundaries(settings.crs)

In [12]:
states = boundaries.states(year=settings.latest)
states.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 56 entries, 0 to 55
Data columns (total 10 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   STATEFP   56 non-null     object  
 1   STATENS   56 non-null     object  
 2   AFFGEOID  56 non-null     object  
 3   GEOID     56 non-null     object  
 4   STUSPS    56 non-null     object  
 5   NAME      56 non-null     object  
 6   LSAD      56 non-null     object  
 7   ALAND     56 non-null     int64   
 8   AWATER    56 non-null     int64   
 9   geometry  56 non-null     geometry
dtypes: geometry(1), int64(2), object(7)
memory usage: 4.5+ KB


<br>
<br>

## Data

<br>

**Attributes**

In [13]:
fields = ['datetimeobject', 'STUSPS', 'positiveIncrease', 'testIncrease', 'deathIncrease', 'hospitalizedIncrease', 
          'positiveCumulative', 'testCumulative',  'deathCumulative', 'hospitalizedCumulative', 
          'positiveRate', 'testRate', 'deathRate', 'hospitalizedRate', 'ndays']

dtype = {'STUSPS': 'str', 'positiveIncrease': np.float64, 'testIncrease': np.float64, 'deathIncrease': np.float64, 
         'hospitalizedIncrease': np.float64, 'positiveCumulative': np.float64, 'testCumulative': np.float64, 
         'deathCumulative': np.float64, 'hospitalizedCumulative': np.float64, 'positiveRate': np.float64, 
         'testRate': np.float64, 'deathRate': np.float64, 'hospitalizedRate': np.float64, 'ndays': np.int64}        

parse_dates = ['datetimeobject']

<br>
<br>

**The URL strings of the sources**

In [14]:
uribaseline = os.path.join(parent, 'warehouse', 'baselines.csv')
urihospitalizations = os.path.join(parent, 'warehouse', 'hospitalizations.csv')

<br>
<br>

**Baseline**

In [15]:
baselines = pd.read_csv(filepath_or_buffer=uribaseline, header=0, usecols=fields, 
                        dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

baselines.loc[:, 'positiveTestRate'] = np.where(
    baselines['testRate'] > 0, 100 * baselines['positiveRate'] / baselines['testRate'], 0)
baselines.loc[:, 'deathPositiveRate'] = np.where(
    baselines['positiveRate'] > 0, 100 * baselines['deathRate'] / baselines['positiveRate'], 0)

logger.info(baselines.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 18148 entries, 0 to 18147
Data columns (total 17 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   datetimeobject          18148 non-null  datetime64[ns]
 1   STUSPS                  18148 non-null  object        
 2   deathIncrease           18148 non-null  float64       
 3   deathCumulative         18148 non-null  float64       
 4   positiveIncrease        18148 non-null  float64       
 5   positiveCumulative      18148 non-null  float64       
 6   testIncrease            18148 non-null  float64       
 7   testCumulative          18148 non-null  float64       
 8   hospitalizedIncrease    18148 non-null  float64       
 9   hospitalizedCumulative  18148 non-null  float64       
 10  deathRate               18148 non-null  float64       
 11  positiveRate            18148 non-null  float64       
 12  testRate                18148 non-null  float6

<br>
<br>

**Hospitalizations**

In [16]:
hospitalizations = pd.read_csv(filepath_or_buffer=urihospitalizations, header=0, 
                               usecols=fields,dtype=dtype, encoding='utf-8', parse_dates=parse_dates)

hospitalizations.loc[:, 'hospitalizedPositiveRate'] =  np.where(
    hospitalizations['positiveRate'] > 0, 100 * hospitalizations['hospitalizedRate'] / hospitalizations['positiveRate'], 0)
hospitalizations.loc[:, 'deathHospitalizedRate'] =  np.where(
    hospitalizations['hospitalizedRate'] > 0, 100 * hospitalizations['deathRate'] / hospitalizations['hospitalizedRate'], 0)

logger.info(hospitalizations.info())

INFO:__main__:None


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12913 entries, 0 to 12912
Data columns (total 17 columns):
 #   Column                    Non-Null Count  Dtype         
---  ------                    --------------  -----         
 0   datetimeobject            12913 non-null  datetime64[ns]
 1   STUSPS                    12913 non-null  object        
 2   deathIncrease             12913 non-null  float64       
 3   deathCumulative           12913 non-null  float64       
 4   positiveIncrease          12913 non-null  float64       
 5   positiveCumulative        12913 non-null  float64       
 6   testIncrease              12913 non-null  float64       
 7   testCumulative            12913 non-null  float64       
 8   hospitalizedIncrease      12913 non-null  float64       
 9   hospitalizedCumulative    12913 non-null  float64       
 10  deathRate                 12913 non-null  float64       
 11  positiveRate              12913 non-null  float64       
 12  testRate          

<br>
<br>

## Regression

<br>

**Latest Date**

In [17]:
latestdate = baselines.datetimeobject.max()
logger.info(type(latestdate))
logger.info(latestdate)

INFO:__main__:<class 'pandas._libs.tslibs.timestamps.Timestamp'>
INFO:__main__:2021-02-20 00:00:00


<br>
<br>

**Periods**

Within the context of the aforementioned $positives_{c, \small{100K}, \tau} \;\; vs.\ \; tests_{c, \small{100K}, \tau}$ curves, each value of this array is the number of data points, starting from the end of a curve, that are used to develop **a regression model of the tangent/secant line to the points**.

Key points:

* The model predicts the expected $positives_{c, \small{100K}}$ as $tests_{c, \small{100K}, \tau}$ increases.

* The gradient of each model is an estimate of the prevailing positivity rate; an estimate of the current daily positivity value. [cf. continous positivity value] 

In [18]:
periods = np.arange(8, 29)
logger.info( periods)

INFO:__main__:[ 8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23 24 25 26 27 28]


<br>
<br>

**Partitions**

In [19]:
partitionby = 'STUSPS'
partitions = baselines[partitionby].unique()
logger.info(partitions)

INFO:__main__:['AL' 'AK' 'AZ' 'AR' 'CA' 'CO' 'CT' 'DE' 'DC' 'FL' 'GA' 'HI' 'ID' 'IL'
 'IN' 'IA' 'KS' 'KY' 'LA' 'ME' 'MD' 'MA' 'MI' 'MN' 'MS' 'MO' 'MT' 'NE'
 'NV' 'NH' 'NJ' 'NM' 'NY' 'NC' 'ND' 'OH' 'OK' 'OR' 'PA' 'RI' 'SC' 'SD'
 'TN' 'TX' 'UT' 'VT' 'VA' 'WA' 'WV' 'WI' 'WY' 'PR']


<br>
<br>

### **positives vs. tests**

<br>

**Analysis**

In [20]:
formula = 'positiveRate ~ testRate'
regressor = 'testRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=100, num=5, latestdate=latestdate)

In [21]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of affairs**

In [22]:
ptr = estimates

<br>

Prevailing Positive Test Rate

In [23]:
ptr.loc[:, 'prevailingPTR'] = 100 * ptr['gradient']
ptr = ptr.merge(baselines[['datetimeobject', 'STUSPS', 'positiveTestRate']], how='left', on=['datetimeobject', 'STUSPS'])
ptr.loc[:, 'cf'] = ptr[['prevailingPTR', 'positiveTestRate']].max(axis=1)

ptr.loc[:, 'rank'] = ptr['cf'].rank(method='min', ascending=False)
ptr.sort_values(by='rank', ascending=True, inplace=True)
ptr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [24]:
ptr.loc[:, 'prevailingTPC'] = np.where(ptr['prevailingPTR'] > 0, ptr['prevailingPTR'].rdiv(100), 0)
ptr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'positiveTestRate': 'continuousPTR'},
          inplace=True)
ptr.to_csv(path_or_buf=os.path.join(warehouse, 'ptr.csv'), index=False, header=True, encoding='utf-8')
ptr.to_json(path_or_buf=os.path.join(warehouse, 'ptr.json'), orient='values')

ptr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingPTR,continuousPTR,rank,prevailingTPC
1,2021-02-20,0.0203809,0.0193097,0.021184,8.53697e-26,0.999099,3004.98,AK,2.03809,3.386783,48.0,49.0655
8,2021-02-20,0.0287988,0.0214863,0.0330784,6.0503e-18,0.99228,750.766,DC,2.87988,3.320924,49.0,34.7237
19,2021-02-20,0.0230845,0.00641027,0.0305275,2.24177e-11,0.943013,576.612,ME,2.30845,2.810988,50.0,43.3191
11,2021-02-20,0.0132414,0.00915645,0.0193113,1.87375e-15,0.975146,979.133,HI,1.32414,2.640671,51.0,75.5207
45,2021-02-20,0.0186119,0.0135353,0.0218902,1.20253e-17,0.98292,-698.876,VT,1.86119,1.422635,52.0,53.729


<br>
<br>

### **deaths vs. positives**

In [25]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [26]:
formula = 'deathRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=baselines, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=20, num=5, latestdate=latestdate)

In [27]:
estimates, predictions = prospects.exc(partitions=baselines[partitionby].unique())

<br>

**State of Affairs**

In [28]:
dpr = estimates

<br>

Prevailing Death Positive Rate

In [29]:
dpr.loc[:, 'prevailingDPR'] = 100 * dpr['gradient']
dpr = dpr.merge(baselines[['datetimeobject', 'STUSPS', 'deathPositiveRate']], how='left', on=['datetimeobject', 'STUSPS'])
dpr.loc[:, 'cf'] = dpr[['prevailingDPR', 'deathPositiveRate']].max(axis=1)

dpr.loc[:, 'rank'] = dpr['cf'].rank(method='min', ascending=False)
dpr.sort_values(by='rank', ascending=True, inplace=True)
dpr.drop(columns=['cf'], inplace=True)

<br>

Structuring

In [30]:
dpr.loc[:, 'prevailingCPD'] = np.where(dpr['prevailingDPR'] > 0, dpr['prevailingDPR'].rdiv(100), 0)
dpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathPositiveRate': 'continuousDPR'},
          inplace=True)
dpr.to_csv(path_or_buf=os.path.join(warehouse, 'dpr.csv'), index=False, header=True, encoding='utf-8')
dpr.to_json(path_or_buf=os.path.join(warehouse, 'dpr.json'), orient='values')

dpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDPR,continuousDPR,rank,prevailingCPD
23,2021-02-20,0.0132226,0.00847376,0.0153964,1.87742e-16,0.984515,1.9134,MN,1.32226,1.343297,48.0,75.628
27,2021-02-20,0.0129666,0.0050884,0.0356856,3.68254e-10,0.93222,-28.1479,NE,1.29666,1.029947,49.0,77.1214
46,2021-02-20,0.0113253,0.00678693,0.0168565,8.13145e-14,0.971505,9.32856,VA,1.13253,1.281040,50.0,88.298
44,2021-02-20,0.00849998,0.00652983,0.0121762,1.08231e-16,0.98763,-39.8181,UT,0.849998,0.503233,51.0,117.647
1,2021-02-20,0.00779693,0.00352617,0.0104758,7.33238e-08,0.865064,-19.1653,AK,0.779693,0.523570,52.0,128.256


<br>
<br>

### **hospitalizations vs. positives**

In [31]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [32]:
formula = 'hospitalizedRate ~ positiveRate'
regressor = 'positiveRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [33]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [34]:
hpr = estimates

<br>

Prevailing Hospitalised Positive Rate

In [35]:
hpr.loc[:, 'prevailingHPR'] = 100 * hpr['gradient']
hpr.loc[:, 'rank'] = hpr['prevailingHPR'].rank(method='min', ascending=False)
hpr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [36]:
hpr = hpr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'hospitalizedPositiveRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
hpr.loc[:, 'prevailingCPH'] = np.where(hpr['prevailingHPR'] > 0, hpr['prevailingHPR'].rdiv(100), 0)

hpr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'hospitalizedPositiveRate': 'continuousHPR'}, 
           inplace=True)

hpr.to_csv(path_or_buf=os.path.join(warehouse, 'hpr.csv'), index=False, header=True, encoding='utf-8')
hpr.to_json(path_or_buf=os.path.join(warehouse, 'hpr.json'), orient='values')

hpr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingHPR,rank,continuousHPR,prevailingCPH
32,2021-02-20,0.0180104,0.00665659,0.0394644,3.16175e-08,0.85681,126.112,MS,1.80104,33.0,3.093281,55.5235
33,2021-02-20,0.0139319,0.0107753,0.0189859,4.21954e-13,0.969459,64.631,AK,1.39319,34.0,2.250082,71.7775
34,2021-02-20,0.00760598,0.005849,0.00862463,2.17429e-17,0.98842,39.4894,NH,0.760598,35.0,1.492680,131.475
35,2021-02-20,0,-2.76197e-14,2.76197e-14,0.900226,-255,462.615,NY,0,36.0,5.724556,0
36,2021-02-20,9.71445e-17,-4.18753e-14,4.23194e-14,0.890364,-144.152,343.787,CT,9.71445e-15,36.0,4.488100,1.02939e+16


<br>
<br>

### **deaths vs. hospitalizations**

In [37]:
del formula, regressor, prospects, estimates, predictions

<br>

**Analysis**

In [38]:
formula = 'deathRate ~ hospitalizedRate'
regressor = 'hospitalizedRate'

prospects = atlantic.investigations.prospects.Prospects(data=hospitalizations, partitionby=partitionby, periods=periods, 
                                                 formula=formula, regressor=regressor, step=10, num=5, latestdate=latestdate)

In [39]:
estimates, predictions = prospects.exc(partitions=hospitalizations[partitionby].unique())

<br>

**State of Affairs**

In [40]:
dhr = estimates

<br>

Prevailing

In [41]:
dhr.loc[:, 'prevailingDHR'] = 100 * dhr['gradient']
dhr.loc[:, 'rank'] = dhr['prevailingDHR'].rank(method='min', ascending=False)
dhr.sort_values(by='rank', ascending=True, inplace=True)

<br>

Structuring

In [42]:
dhr = dhr.merge(hospitalizations[['datetimeobject', 'STUSPS', 'deathHospitalizedRate']], 
                how='left', on=['datetimeobject', 'STUSPS'])
dhr.loc[:, 'prevailingHPD'] = np.where(dhr['prevailingDHR'] > 0, dhr['prevailingDHR'].rdiv(100), 0)

dhr.rename(columns={'lowerconfidenceinterval': 'gradientLCI', 'upperconfidenceinterval': 'gradientUCI', 
                    'deathHospitalizedRate': 'continuousDHR'}, 
           inplace=True)

dhr.to_csv(path_or_buf=os.path.join(warehouse, 'dhr.csv'), index=False, header=True, encoding='utf-8')
dhr.to_json(path_or_buf=os.path.join(warehouse, 'dhr.json'), orient='values')

dhr.tail()

,datetimeobject,gradient,gradientLCI,gradientUCI,pvalue,rsquared,intercept,STUSPS,prevailingDHR,rank,continuousDHR,prevailingHPD
32,2021-02-20,0.20325,0.162012,0.249374,6.52901e-15,0.982377,20.5336,CO,20.325,33.0,25.423216,4.92005
33,2021-02-20,0.20031,0.159019,0.542037,6.67492e-11,0.980172,40.9024,MT,20.031,34.0,29.687846,4.99226
34,2021-02-20,0.191722,0.126986,0.226527,1.89203e-16,0.986118,27.4518,MN,19.1722,35.0,25.261543,5.21589
35,2021-02-20,0.178463,0.0272897,0.464184,4.76528e-07,0.835747,2.82483,HI,17.8463,36.0,19.706691,5.60342
36,2021-02-20,0.17626,0.12677,0.2488,4.09508e-16,0.985394,-22.0467,UT,17.626,37.0,12.773039,5.67345


<br>
<br>

## Clean-up

In [43]:
!rm -rf *.log
!rm -rf *.pdf
!rm -rf states

<br>
<br>

## End

In [44]:
%%bash

date +"%Y-%m-%d %T"

2021-02-21 18:19:04
